# Camera calibration

Notebook is based on OpenCV tutorial (https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)


see also for visualization:

https://www.rerun.io/


online viewer:

https://www.rerun.io/viewer


## Preliminary code

In [ ]:
import cv2
import logging
import io
import numpy
import plotly.express as plte


logger = logging.getLogger(__name__)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def load_image(payload: any) -> any:
    np_image = numpy.frombuffer(payload, numpy.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    return img


def find_chess(image: any, corners_x: int, corners_y: int) -> numpy.ndarray:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (corners_x, corners_y), None)

    # If found, add object points, image points (after refining them)
    if ret != True:
        logger.error("Error, grid (%d, %d) is not found in image",
                     corners_x, corners_y)
        return None

    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)

    return corners2


def get_chess_corners_from_image(
        image: any, corners_x: int, corners_y: int) -> tuple[any, any]:
    corners = find_chess(image=image, corners_x=corners_x,
                         corners_y=corners_y)
    if corners is None:
        return None, image
    result_image = cv2.drawChessboardCorners(image, (corners_x, corners_x),
                                             corners, True)
    return corners, result_image


def undistort_image(image: numpy.ndarray, mtx: numpy.ndarray,
                    dist: numpy.ndarray, new_mtx: numpy.ndarray,
                    canvas_w: int, canvas_h: int) -> numpy.ndarray:
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, new_mtx,
                                             (canvas_w, canvas_h), 5)
    return cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)


In [ ]:
import pandas
import numpy
import logging
import functools


logger = logging.getLogger(__name__)


def load_single_image(item: pandas.Series, url_template: str) -> numpy.ndarray:
    item_file = item["image"].lstrip(" ")
    url = url_template.format(item_file)
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item_file, url)
    return pandas.Series([load_image(response.content), url],
                         index=["image_data", "url"])

In [ ]:
import requests
import logging


logger = logging.getLogger(__name__)


def match_points(
        corners_x: int,
        corners_y: int,
        corners: numpy.ndarray,
) -> tuple[numpy.ndarray, numpy.ndarray]:
    objp = numpy.zeros((corners_x*corners_y, 3), numpy.float32)
    objp[:,:2] = numpy.mgrid[0:corners_x, 0:corners_y].T.reshape(-1, 2)
    return objp


def calculate_image_corners(
        row: pandas.Series, ignore_list: str=None, shapes: numpy.ndarray=None,
        image_field="image_data"
) -> pandas.Series:
    result = pandas.Series({
        "corners": None, "chess_image": None, "object_points": None})
    if ignore_list is not None and row.image in ignore_list:
        return result

    corners, image = get_chess_corners_from_image(
        image=numpy.copy(row[image_field]),
        corners_x=int(row.corners_x),
        corners_y=int(row.corners_y)
    )

    img_shape = (row.image_data.shape[1], row.image_data.shape[0])

    if corners is None:
        objp = None
        logger.error("Can't find %d, %d conrenrs in %s",
                     row.corners_x, row.corners_y, row.image)
    else:
        objp = match_points(corners_x=int(row.corners_x),
                            corners_y=int(row.corners_y),
                            corners=corners)

    result.corners = corners
    result.chess_image = image
    result.object_points= objp

    return result


In [ ]:
import logging


logger = logging.getLogger(__name__)


def undistort_image_from_row(
        row: pandas.Series,
        camera_matrix: numpy.ndarray,
        distortion_matrix: numpy.ndarray,
        new_camera_matrix: numpy.ndarray,
        region: tuple[int, int, int, int]
) -> pandas.Series:
    axis = numpy.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1, 3)
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    result = pandas.Series({"undistorted": None, "undistorted_debug": None,
                            "reconstructed_points": None, "imgpoints": None})
    if row["corners"] is None or row["corners"] is numpy.nan:
        logger.warning("Image %s has no detected chessboard corners, skipping...",
                       row["image"])
        return result

    pre_dst = undistort_image(
        image=row["image_data"],
        mtx=camera_matrix, dist=distortion_matrix,
        new_mtx=new_camera_matrix,
        canvas_w=row["image_data"].shape[1], canvas_h=row["image_data"].shape[0])
    # crop the image
    x, y, w, h = region
    dst = numpy.copy(pre_dst) # [y:y + h, x:x + w]
    imgpoints, _ = cv2.projectPoints(
        row["object_points"], row["rvec"], row["tvec"],
        camera_matrix, distortion_matrix)
    error = cv2.norm(row["corners"], imgpoints, cv2.NORM_L2)/len(imgpoints)
    lwidth = 2
    dst = cv2.line(dst, (x, y), (x + w, y), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y), (x + w, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y + h), (x, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x, y + h), (x, y), (0, 255, 0), lwidth)

    result["undistorted_debug"] = dst
    result["undistorted"] = pre_dst
    result["reconstructed_points"] = imgpoints
    result["error"] = error

    return result

def draw_image_sequence(*args, axis=1) -> numpy.ndarray:
    if len(args) == 0:
        return numpy.ndarray()

    w = args[0].shape[1]
    h = args[0].shape[0]
    canvas_shape = list(args[0].shape)
    canvas_shape[axis] = canvas_shape[axis]*len(args)
    canvas = numpy.zeros(canvas_shape)
    for i in range(0, len(args)):
        if axis == 1:
            canvas[:, i*w:(i + 1)*w, :] = args[i]
        else:
            canvas[i*w:(i + 1)*w, :, :] = args[i]

    return canvas

In [ ]:
def calculate_fundamental_matrix(
    p1: numpy.ndarray, p2: numpy.ndarray
) -> tuple[numpy.ndarray, numpy.ndarray]:
    p1 = numpy.int32(p1)
    p2 = numpy.int32(p2)
    return cv2.findFundamentalMat(p1, p2, cv2.FM_LMEDS)


def draw_matched_points(
    left_image: numpy.ndarray,
    right_image: numpy.ndarray,
    left_points: numpy.ndarray,
    right_points: numpy.ndarray
) -> None:
    canvas_shape = list(left_image.shape)
    canvas_shape[1] = canvas_shape[1]*2
    canvas = numpy.zeros(canvas_shape)
    canvas[:, 0:left_image.shape[1], :] = left_image
    canvas[:, left_image.shape[1]:, :] = right_image
    for p_left, p_right in zip(left_points, right_points):
        color = tuple(numpy.random.randint(0, 255, 3).tolist())
        left_point = tuple(map(int, p_left))
        canvas = cv2.circle(canvas, left_point, 5, color,-1)
        p_right_shifted = list(p_right)
        p_right_shifted[0] = p_right_shifted[0] + left_image.shape[1]
        right_point = tuple(map(int, p_right_shifted))
        canvas = cv2.circle(canvas, right_point, 5, color,-1)
        canvas = cv2.line(canvas, left_point, right_point, (0, 255, 0), 1)
    fig = plte.imshow(canvas)
    fig.show()

In [ ]:
def calculate_norm(vector: numpy.ndarray) -> float:
    return numpy.linalg.norm(vector, ord=2)


def calculate_angle(a_vector: numpy.ndarray, b_vector: numpy.ndarray) -> float:
    a_norm = calculate_norm(a_vector)
    b_norm = calculate_norm(b_vector)
    return numpy.arccos(sum(a_vector * b_vector)/(a_norm * b_norm))*180/numpy.pi



In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt


def calculate_sift_descriptors(image: any) -> tuple[any, any]:
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    return sift.detectAndCompute(image, None)


def calculate_orb_descriptors(image: numpy.ndarray) -> tuple[any, any]:
    orb = cv.ORB_create()
    # find the keypoints with ORB
    kp = orb.detect(image, None)
    # compute the descriptors with ORB
    kp, des = orb.compute(image, kp)

    return kp, des


def calculate_match_points(
        kp1: numpy.ndarray,
        des1: numpy.ndarray,
        kp2: numpy.ndarray,
        des2: numpy.ndarray) -> tuple[list, list]:
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    return pts1, pts2

## Data preparation

In [ ]:
!wget -O samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/position.csv

In [ ]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/{}"
ignore_list = [
    "image75_r.jpg", "image84_r.jpg",
    "image84_l.jpg", "image89_r.jpg",
    "image89_l.jpg", "image91_r.jpg",
    "image91_l.jpg", "image36_r.jpg",
    "image36_l.jpg"
]

dataframe = pandas.read_csv("samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)

## Single camera calibration

In [ ]:
# %debug
shapes = dataframe["image_data"].apply(lambda x: x.shape).unique()
if shapes.shape[0] != 1:
    logger.error("More than one unique image sizes were found: %s",
                 str(shapes))
shapes = (shapes[0][1], shapes[0][0])

filtered = dataframe[~dataframe["image"].isin(ignore_list)]
dataframe = dataframe.join(
    filtered.apply(
        functools.partial(
            calculate_image_corners,
            ignore_list=ignore_list,
            shapes=None),
        axis=1)
)

filtered = dataframe[~dataframe["image"].isin(ignore_list)]

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    filtered["object_points"].tolist(),
    filtered["corners"].tolist(), shapes,
    None, None)
filtered = filtered.assign(rvec=rvecs, tvec=tvecs)
dataframe = dataframe.join(filtered[["rvec", "tvec"]])

w = shapes[0]
h = shapes[1]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
(roi, newcameramtx)

In [ ]:
filtered["object_points"]

In [ ]:
# %debug
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            undistort_image_from_row,
            camera_matrix=mtx,
            distortion_matrix=dist,
            new_camera_matrix=newcameramtx,
            region=roi
        ), axis=1
    )
)
x, y, w, h = roi
dataframe.undistorted = dataframe.apply(
    lambda row: row.undistorted[
        y:y + h, x:x + w] if row.undistorted is not None else None
    , axis=1)
dataframe[["image", "error"]]

filtered_undistorted = dataframe[~dataframe["image"].isin(ignore_list)]
dataframe = dataframe.join(
    filtered_undistorted.apply(functools.partial(
        calculate_image_corners, image_field="undistorted"
    ), axis=1), rsuffix="_undistorted")
dataframe.columns

In [ ]:
plte.imshow(dataframe.chess_image.iloc[0]).show()
plte.imshow(dataframe.undistorted.iloc[0]).show()

## Stereo pair calibration

[stereoCalibrate](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga9d2539c1ebcda647487a616bdf0fc716) - calculates transformations to locate points of one camera's image on another one based on known pattern.

[stereoRectify](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga617b1685d4059c6040827800e72ad2b6) - calculates a transformation to represent images from stereo pair on a single plane.

[C++ example](https://github.com/LiliMeng/Build3dModel/blob/master/build3dmodel.cpp) - stereo camera calibration.

[C++ example](https://github.com/jhicks256/OpenCV-Samples/blob/ab96b66f15bca8798c56e6ce0adf8b31ba957616/stereo_match.cpp#L202) - stereo images match.

### Stereo calibration and depth estimation

cameraCalibration --> stereoCalibtation --> stereoRectify

In [ ]:
stereodata = dataframe[dataframe.type == "r"].merge(
    dataframe[dataframe.type == "l"], on="no", suffixes=["_right", "_left"])
stereodata.columns

In [ ]:
stereo_calib_data = stereodata[
    (~stereodata.corners_left.isnull()) & (~stereodata.corners_right.isnull())]
stereo_calib_data = stereo_calib_data[
    (stereo_calib_data.object_points_left.apply(lambda x: x.shape)
     ==stereo_calib_data.object_points_right.apply(lambda x: x.shape))
]
len(stereo_calib_data)

In [ ]:
stereocalibration_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
scalib_results = pandas.Series(cv2.stereoCalibrate(
    numpy.array([stereo_calib_data.object_points_left.iloc[0]]),
    numpy.array([stereo_calib_data.corners_left.iloc[0]]),
    numpy.array([stereo_calib_data.corners_right.iloc[0]]),
    cameraMatrix1=mtx,
    cameraMatrix2=mtx,
    distCoeffs1=dist,
    distCoeffs2=dist,
    imageSize=shapes,
    criteria=stereocalibration_criteria, flags = stereocalibration_flags
), index=["error", "camera_matrix_left", "dist_coeffs_left",
          "camera_matrix_right", "dist_coeffs_right", "R", "T", "E", "F"])
scalib_results

In [ ]:
srect_results = pandas.Series(cv2.stereoRectify(
    cameraMatrix1=scalib_results.camera_matrix_left,
    distCoeffs1=scalib_results.dist_coeffs_left,
    cameraMatrix2=scalib_results.camera_matrix_right,
    distCoeffs2=scalib_results.dist_coeffs_right,
    imageSize=shapes,
    R=scalib_results.R, T=scalib_results["T"],
    flags=cv2.CALIB_ZERO_DISPARITY
), index=["R1", "R2", "P1", "P2", "Q", "validPixROI1", "validPixROI"])
srect_results

### Test stereo image rectification

In [ ]:
left_map1, left_map2 = cv2.initUndistortRectifyMap(
    cameraMatrix=scalib_results.camera_matrix_left,
    distCoeffs=scalib_results.dist_coeffs_left,
    R=srect_results.R1,
    newCameraMatrix=srect_results.P1, size=shapes, m1type=cv2.CV_16SC2) # , rmap[0][0], rmap[0][1]);
right_map1, right_map2 = cv2.initUndistortRectifyMap(
    cameraMatrix=scalib_results.camera_matrix_right,
    distCoeffs=scalib_results.dist_coeffs_right,
    R=srect_results.R2,
    newCameraMatrix=srect_results.P2, size=shapes, m1type=cv2.CV_16SC2
)

In [ ]:
left_rimage = cv2.remap(
    stereo_calib_data.image_data_left.iloc[0],
    left_map1, left_map2, cv2.INTER_LINEAR)
right_rimage = cv2.remap(
    stereo_calib_data.image_data_right.iloc[0],
    right_map1, right_map2, cv2.INTER_LINEAR)

### Depth estimation with SIFT algorithm

In [ ]:
kp1, des1 = calculate_sift_descriptors(image=left_rimage)
kp2, des2 = calculate_sift_descriptors(image=right_rimage)

left_pts, right_pts = calculate_match_points(
    kp1=kp1, des1=des1,
    kp2=kp2, des2=des2)
draw_matched_points(
    left_image=left_rimage,
    right_image=right_rimage,
    left_points=left_pts,
    right_points=right_pts
)

In [ ]:
Fm, mask = calculate_fundamental_matrix(
    p1=left_pts, p2=right_pts
)
left_pts_filtered = numpy.array(left_pts)[mask.ravel()==1]
right_pts_filtered = numpy.array(right_pts)[mask.ravel()==1]

draw_matched_points(
    left_image=left_rimage,
    right_image=right_rimage,
    left_points=left_pts_filtered,
    right_points=right_pts_filtered
)
print(len(left_pts_filtered))

### Depth estimation with ORB descriptors

In [ ]:
kp1, des1 = calculate_orb_descriptors(image=left_rimage)
kp2, des2 = calculate_orb_descriptors(image=right_rimage)

left_pts, right_pts = calculate_match_points(
    kp1=kp1, des1=des1.astype(numpy.float32),
    kp2=kp2, des2=des2.astype(numpy.float32))
draw_matched_points(
    left_image=left_rimage,
    right_image=right_rimage,
    left_points=left_pts,
    right_points=right_pts
)

### Depth estimation with manual points

Below few points were manualy matched between left/right image to calculate disparity and estimate depth by Q matrix that was calculated for calibrated stereo pair.

In [ ]:
pts1 = numpy.array([[385.0, 624.0, 533.0, 535.0, 261.0, 416.0],
                    [28.0, 38.0, 211.0, 233.0, 0.0, 245.0]])
pts2 = numpy.array([[93.0, 304.0, 146.0, 120.0, 257.0, 1054.0-639.0],
                    [28.0, 38.0, 211.0, 233.0, 0.0, 242.0]])
disp = pts1[0,:] - pts2[0, :]
pts1, pts2, disp

In [ ]:
draw_matched_points(
    left_image=left_rimage,
    right_image=right_rimage,
    left_points=pts1.swapaxes(0, 1),
    right_points=pts2.swapaxes(0, 1)
)

Below 1 value was added to each point for compartability with scaling representation.

In [ ]:
pts = numpy.concatenate((pts1, disp.reshape(1, -1), numpy.ones((1, pts1.shape[1]))))
pts, disp

Calculates 3D from 2D in scaled dimension

In [ ]:
points4d = cv2.triangulatePoints(projMatr1=srect_results.P1,
                                 projMatr2=srect_results.P2,
                                 projPoints1=pts1,
                                 projPoints2=pts2)
points4d

Removes scaling

In [ ]:
points = (points4d[:-1, :]/points4d[-1, :]).swapaxes(0, 1)
points

In [ ]:
actual_points = numpy.array(
    [[0.0, 0.0, 0.0], [6.0, 0.0, 0.0], [6.0, 8.0, 0.0]]
)

Checks hypotesis for three points on checkbord (6x8x10 triangle + single 90 and two 45 angles between its sides)

In [ ]:
ab_vector = points[1] - points[0]
ab_norm = numpy.linalg.norm(ab_vector, ord=2)
bc_vector = points[2] - points[1]
bc_norm = numpy.linalg.norm(bc_vector, ord=2)
ac_vector = points[2] - points[0]
ac_norm = numpy.linalg.norm(ac_vector, ord=2)

actual_ab_vector = actual_points[1] - actual_points[0]
actual_ab_norm = numpy.linalg.norm(actual_ab_vector, ord=2)
actual_bc_vector = actual_points[2] - actual_points[1]
actual_bc_norm = numpy.linalg.norm(actual_bc_vector, ord=2)
actual_ac_vector = actual_points[2] - actual_points[0]
actual_ac_norm = numpy.linalg.norm(actual_ac_vector, ord=2)

In [ ]:
{
    "estimated": (ab_norm, bc_norm, ac_norm),
    "actual": (actual_ab_norm, actual_bc_norm, actual_ac_norm)
}

Angles are calculated from vectors scalar value:

$\overrightarrow a \cdot \overrightarrow b = |a| |b| cos(\angle ab)$

$\angle AB,BC$

In [ ]:
{
    "estimated": calculate_angle(ab_vector, bc_vector),
    "actual": calculate_angle(actual_ab_vector, actual_bc_vector)
}

$\angle AB,AC$

In [ ]:
{
    "estimated": calculate_angle(ab_vector, ac_vector),
    "actual": calculate_angle(actual_ab_vector, actual_ac_vector)
}

$\angle AC,BC$

In [ ]:
{
    "estimated": calculate_angle(ac_vector, bc_vector),
    "actual": calculate_angle(actual_ac_vector, actual_bc_vector)
}

### Depth estimation with out of the box algorithm (block match)

In [ ]:
stereo = cv2.StereoBM.create(numDisparities=256, blockSize=5)
left_rgray = cv2.cvtColor(left_rimage, cv2.COLOR_BGR2GRAY)
right_rgray = cv2.cvtColor(right_rimage, cv2.COLOR_BGR2GRAY)
disparity = stereo.compute(left_rgray, right_rgray)
plte.imshow(disparity).show()

## Web camera experiments

In [ ]:
!wget -O real-samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-real-test/position.csv

In [ ]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-real-test/{}"
dataframe = pandas.read_csv("real-samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)
plte.imshow(draw_image_sequence(dataframe.image_data.iloc[1], dataframe.image_data.iloc[0]))

In [ ]:
left_image = dataframe[dataframe["type"] == "l"]["image_data"].iloc[0]
right_image = dataframe[dataframe["type"] == "r"]["image_data"].iloc[0]

In [ ]:
kp1, des1 = calculate_sift_descriptors(image=left_image)
kp2, des2 = calculate_sift_descriptors(image=right_image)

left_pts, right_pts = calculate_match_points(
    kp1=kp1, des1=des1.astype(numpy.float32),
    kp2=kp2, des2=des2.astype(numpy.float32))
draw_matched_points(
    left_image=left_image,
    right_image=right_image,
    left_points=left_pts,
    right_points=right_pts
)

In [ ]:
Fm, mask = calculate_fundamental_matrix(
    p1=left_pts, p2=right_pts
)
left_pts_filtered = numpy.array(left_pts)[mask.ravel()==1]
right_pts_filtered = numpy.array(right_pts)[mask.ravel()==1]

In [ ]:
draw_matched_points(
    left_image=left_image,
    right_image=right_image,
    left_points=left_pts_filtered,
    right_points=right_pts_filtered
)